In [5]:
# All imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import random

In [6]:
# Initialize node class
class Node:
    
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None
        self.name = -1


In [97]:
# Implement a decision tree classifier
class MyDecisionTreeClassifier:
    
    def __init__(self, max_depth, features):
        self.max_depth = max_depth
        self.root = None
        self.features = features
    
    def gini(self, group):
        '''
        A Gini score gives an idea of how good a split is by how mixed the
        classes are in the two groups created by the split.
        
        A perfect separation results in a Gini score of 0,
        whereas the worst case split that results in 50/50
        classes in each group result in a Gini score of 0.5
        (for a 2 class problem).
        '''
        if len(group) == 0:
            return 1
        gini = 1
        for clas in [i for i in range(self.features)]:
            gini -= (group.count(clas)/len(group))**2
        return round(gini, 4)
    
    def split_data(self, X, y) -> tuple[int, int]:
        
        # test all the possible splits in O(N*F) where N in number of samples
        # and F is number of features

        index = -1
        threshhold = -1
        best_ig = float('inf')

        # iterate over every threshold in dataset
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                group1 = [y[k] for k, elem in enumerate(X[:,j]) if elem <= X[i,j]]
                group2 = [y[k] for k, elem in enumerate(X[:,j]) if elem > X[i,j]]
                gini1 = self.gini(group1)
                gini2 = self.gini(group2)

                # relative ig, because root gini is constant
                ig = (len(group1)/len(y))*gini1 + (len(group2)/len(y))*gini2
                if ig < best_ig:
                    best_ig = ig
                    threshhold = X[i,j]
                    index = j
        
        return (index, threshhold)
    
    def build_tree(self, X, y, depth = 0):

        # at first we set three indicators when our tree should stop building itself (exit recursion)
        if list(y) == []:
            return None
        if len(set(y)) == 1:
            leaf = Node(X, y)
            leaf.right = None
            leaf.left = None
            leaf.name = -1
            for elem in set(list(y)):
                if list(y).count(elem) > list(y).count(leaf.name):
                    leaf.name = elem 
            return leaf
        if depth == self.max_depth:
            leaf = Node(X, y)
            leaf.right = None
            leaf.left = None
            leaf.name = -1
            for elem in set(list(y)):
                if list(y).count(elem) > list(y).count(leaf.name):
                    leaf.name = elem 
            return leaf
        
        # now we create a root node
        root = Node(X, y)
        for elem in set(list(y)):
                if list(y).count(elem) > list(y).count(root.name):
                    root.name = elem 
        split = self.split_data(X, y)
        root.feature_index = split[0]
        root.threshold = split[1]

        # based on split we calculate data for right and left child
        y_left = np.array([y[k] for k, elem in enumerate(X[:,root.feature_index]) if elem <= root.threshold])
        y_right = np.array([y[k] for k, elem in enumerate(X[:,root.feature_index]) if elem > root.threshold])
        X_left = np.array([X[k] for k, elem in enumerate(X[:,root.feature_index]) if elem <= root.threshold])
        X_right = np.array([X[k] for k, elem in enumerate(X[:,root.feature_index]) if elem > root.threshold])

        # recursively make a tree
        root.left = self.build_tree(X_left, y_left, depth+1)
        root.right = self.build_tree(X_right, y_right, depth+1)
        return root

        
    
    def fit(self, X, y):
        # basically wrapper for build tree / train
        self.root = self.build_tree(X,y)
        return 0

    def predict(self, root, X_test):
        
        # traverse the tree while there is a child
        # and return the predicted class for it, 
        if X_test[root.feature_index] > root.threshold:
            if root.right != None:
                return self.predict(root.right, X_test)
            return root.name
        if X_test[root.feature_index] <= root.threshold:
            if root.left != None:
                return self.predict(root.left, X_test)
            return root.name
        
    def evaluate(self, X_test, y_test):
        # evaluate success rate
        plus = 0
        for i, x in enumerate(list(X_test)):
            if self.predict(self.root, x) == y_test[i]:
                plus += 1
        return f'{round((plus/len(X_test))*100, 2)}'

In [11]:
#load data
breast = load_breast_cancer()
iris = load_iris()

In [162]:
treee = MyDecisionTreeClassifier(6,2)
X, y = breast.data, breast.target

# build a tree 10 times and print avarage success rate for breast cancer dataset

sum = 0
for _ in range(10):
    X, y = shuffle(X, y, random_state=random.randint(0, 10000000))
    train_X = X[:int(round(len(X)*0.8))]
    train_y = y[:int(round(len(X)*0.8))]
    test_X = X[int(round(len(X)*0.8)):]
    test_y = y[int(round(len(X)*0.8)):]
    treee.fit(train_X, train_y)
    sum += float(treee.evaluate(test_X, test_y))

print(f'{round(sum/10, 2)}%')

92.54%


In [148]:
treee = MyDecisionTreeClassifier(6, 3)
X, y = iris.data, iris.target

# build a tree 50 times and print avarage success rate for iris dataset

sum = 0
for _ in range(50):
    X, y = shuffle(X, y, random_state=random.randint(0, 10000000))
    train_X = X[:int(round(len(X)*0.8))]
    train_y = y[:int(round(len(X)*0.8))]
    test_X = X[int(round(len(X)*0.8)):]
    test_y = y[int(round(len(X)*0.8)):]
    treee.fit(train_X, train_y)
    sum += float(treee.evaluate(test_X, test_y))

print(f'{round(sum/50, 2)}%')

95.07%
